In [246]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import re

In [247]:
source = requests.get('http://ufcstats.com/statistics/events/completed?page=all').text
soup = BeautifulSoup(source, 'lxml')

In [248]:
cards = soup.find('table')

In [249]:
body = cards.find('tbody')

In [250]:
rows = body.find_all('tr')[2:]

In [251]:
def get_url_list(rows):
    
    url_list = []
    for row in rows:
        link = row.find_all('a')[0].get('href')
        url_list.append(link)
    
    return url_list  

In [252]:
event_list = get_url_list(rows)

In [253]:
headline = []
fight_url = []
winner = []
winner_url = []
loser = []
loser_url = []
winner_kd = []
loser_kd = []
winner_str = []
loser_str = []
winner_td = []
loser_td = []
winner_sub = []
loser_sub = []
weight_class = []
win_method = []
round_num = []
time = []

In [254]:
def get_event_info(url):
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    event_title = soup.find('span', class_= 'b-content__title-highlight').get_text().strip()
    
    return event_title

In [255]:
def get_fight_info(url):
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
    fights = soup.find('table', class_='b-fight-details__table b-fight-details__table_style_margin-top b-fight-details__table_type_event-details js-fight-table')
    table_body = fights.find('tbody', class_='b-fight-details__table-body')
    bouts = table_body.find_all('tr')
    
    return bouts

In [257]:
def get_stats(bouts, event_title):
    
    for bout in bouts:
        headline.append(event_title)
        sections = bout.find_all('td')
        fight_url.append(sections[0].find_all('a')[0].get('href').strip())
        winner.append(sections[1].find_all('a')[0].get_text().strip())
        winner_url.append(sections[1].find_all('a')[0].get('href').strip())
        loser.append(sections[1].find_all('a')[1].get_text().strip())
        loser_url.append(sections[1].find_all('a')[1].get('href').strip())
        winner_kd.append(sections[2].find_all('p')[0].get_text().strip())
        loser_kd.append(sections[2].find_all('p')[1].get_text().strip())
        winner_str.append(sections[3].find_all('p')[0].get_text().strip())
        loser_str.append(sections[3].find_all('p')[1].get_text().strip())
        winner_td.append(sections[4].find_all('p')[0].get_text().strip())
        loser_td.append(sections[4].find_all('p')[1].get_text().strip())
        winner_sub.append(sections[5].find_all('p')[0].get_text().strip())
        loser_sub.append(sections[5].find_all('p')[1].get_text().strip())
        weight_class.append(sections[6].get_text().strip())
        win_method.append(sections[7].find_all('p')[0].get_text().strip())
        round_num.append(sections[8].get_text().strip())
        time.append(sections[9].get_text().strip())

In [258]:
def crawler(url_list):
    for url in url_list:
        event_name = get_event_info(url)
        matchups = get_fight_info(url)
        final_data = get_stats(matchups, event_name)
    
    final_data = pd.DataFrame([headline, fight_url, winner, winner_url, loser, loser_url, winner_kd, loser_kd, winner_str, loser_str, winner_td, loser_td, winner_sub, loser_sub, weight_class, win_method, round_num, time])
    final_data = final_data.transpose()
    final_data.columns = ['Headline', 'Fight URL', 'Winner', 'Winner URL', 'Loser', 'Loser URL', 'Winner KD', 'Loser KD', 'Winner STR', 'Loser STR', 'Winner TD', 'Loser TD', 'Winner Sub', 'Loser Sub', 'Weight Class', 'Win Method', 'Round Num', 'Time']
    
    return final_data

In [259]:
data = crawler(event_list)

In [260]:
data

,Headline,Fight URL,Winner,Winner URL,Loser,Loser URL,Winner KD,Loser KD,Winner STR,Loser STR,Winner TD,Loser TD,Winner Sub,Loser Sub,Weight Class,Win Method,Round Num,Time
0,UFC 276: Adesanya vs. Cannonier,http://ufcstats.com/fight-details/a7eaf7b10116...,Israel Adesanya,http://ufcstats.com/fighter-details/1338e2c748...,Jared Cannonier,http://ufcstats.com/fighter-details/13a0275fa1...,0,0,116,90,0,0,0,0,Middleweight,U-DEC,5,5:00
1,UFC 276: Adesanya vs. Cannonier,http://ufcstats.com/fight-details/4047e9813230...,Alexander Volkanovski,http://ufcstats.com/fighter-details/e124894134...,Max Holloway,http://ufcstats.com/fighter-details/150ff4cc64...,0,0,199,127,0,0,0,0,Featherweight,U-DEC,5,5:00
2,UFC 276: Adesanya vs. Cannonier,http://ufcstats.com/fight-details/0308bcdd00cc...,Alex Pereira,http://ufcstats.com/fighter-details/e5549c82bf...,Sean Strickland,http://ufcstats.com/fighter-details/0d8011111b...,1,0,25,15,0,0,0,0,Middleweight,KO/TKO,1,2:36
3,UFC 276: Adesanya vs. Cannonier,http://ufcstats.com/fight-details/12a33d16e1d4...,Bryan Barberena,http://ufcstats.com/fighter-details/a331233f59...,Robbie Lawler,http://ufcstats.com/fighter-details/f2925e6db4...,0,0,147,81,0,0,0,0,Welterweight,KO/TKO,2,4:47
4,UFC 276: Adesanya vs. Cannonier,http://ufcstats.com/fight-details/39dccca8e808...,Pedro Munhoz,http://ufcstats.com/fighter-details/6bd0211959...,Sean O'Malley,http://ufcstats.com/fighter-details/b50a426a33...,0,0,26,25,0,0,0,0,Bantamweight,CNC,2,3:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6689,UFC 2: No Way Out,http://ufcstats.com/fight-details/3b020d4914b4...,Orlando Wiet,http://ufcstats.com/fighter-details/a6c2f5381d...,Robert Lucarelli,http://ufcstats.com/fighter-details/96087e90d9...,0,0,8,2,0,1,0,1,Open Weight,KO/TKO,1,2:50
6690,UFC 2: No Way Out,http://ufcstats.com/fight-details/d917c8c7461b...,Frank Hamaker,http://ufcstats.com/fighter-details/c3c23c9947...,Thaddeus Luster,http://ufcstats.com/fighter-details/505934897b...,0,0,2,0,1,0,3,0,Open Weight,SUB,1,4:52
6691,UFC 2: No Way Out,http://ufcstats.com/fight-details/ccee020be2e8...,Johnny Rhodes,http://ufcstats.com/fighter-details/319fa1bd31...,David Levicki,http://ufcstats.com/fighter-details/49590e0508...,0,0,11,4,1,0,0,0,Open Weight,KO/TKO,1,12:13
6692,UFC 2: No Way Out,http://ufcstats.com/fight-details/4b9ae533ccb3...,Patrick Smith,http://ufcstats.com/fighter-details/46c8ec317a...,Ray Wizard,http://ufcstats.com/fighter-details/ea0ad15545...,0,0,1,1,0,0,1,0,Open Weight,SUB,1,0:58


In [261]:
data.to_csv('ufc_fights.csv', index=False)